In [18]:
import sys
sys.path.append('../')

from src.cmesrc.config import CMESRCV2_DB, LASCO_CME_DATABASE
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()
cme_id_pa = pd.read_sql("""
SELECT C.cme_id, C.cme_pa FROM CMES C
""", conn)
cme_id_pa.head()

,cme_id,cme_pa
0,19960111001436,267.0
1,19960113220830,265.0
2,19960115070110,262.0
3,19960122031101,105.0
4,19960126091619,90.0


In [3]:
import numpy as np

old_ids = cme_id_pa["cme_id"].astype(int).to_numpy()

# Need to choose cme_pa[:-2] for each row
new_ids = cme_id_pa["cme_id"].astype(str).to_numpy() + np.array([f"{int(i):03}" if not np.isnan(i) else "999" for i in cme_id_pa["cme_pa"]])

new_ids

array(['19960111001436267', '19960113220830265', '19960115070110262', ...,
       '20220831110006227', '20220831141205227', '20220831174805228'],
      dtype=object)

In [4]:
# Check all ids are unique
assert len(new_ids) == len(set(new_ids))

In [5]:
# Now create translation dictionary

translation_dict = {
    old_id : new_id for old_id, new_id in zip(old_ids, new_ids)
}

In [14]:
conn = sqlite3.connect(CMESRCV2_DB)
cursor = conn.cursor()

cur.executescript("""
CREATE INDEX IF NOT EXISTS idx_cmes_cme_id ON CMES(cme_id);
CREATE INDEX IF NOT EXISTS idx_chsc_cme_id ON CMES_HARPS_SPATIALLY_CONSIST(cme_id);
CREATE INDEX IF NOT EXISTS idx_che_cme_id ON CMES_HARPS_EVENTS(cme_id);
CREATE INDEX IF NOT EXISTS idx_fcha_cme_id ON FINAL_CME_HARP_ASSOCIATIONS(cme_id);
CREATE INDEX IF NOT EXISTS idx_ms_cme_id ON MAJUMDAR_SRC(cme_id);
CREATE INDEX IF NOT EXISTS idx_hbb_pres_prev_cme_id ON HARPS_BBOX(prev_present_at_cme_id);
CREATE INDEX IF NOT EXISTS idx_hbb_pres_next_cme_id ON HARPS_BBOX(next_present_at_cme_id);
""")

conn.commit()
conn.close()

In [15]:
import sqlite3

# Connect to your SQLite database
conn = sqlite3.connect(CMESRCV2_DB)
cursor = conn.cursor()

# Step 1: Disable foreign key constraints
cursor.execute("PRAGMA foreign_keys = OFF;")

# Step 3: Update cme_id values in CMES table
for old_id, new_id in translation_dict.items():
    cursor.execute(f"UPDATE CMES SET cme_id = ? WHERE cme_id = ?", (int(new_id), int(old_id)))

# Step 4: Update foreign key references in all other tables
tables_to_update = ['CMES_HARPS_SPATIALLY_CONSIST', 'CMES_HARPS_EVENTS', 'FINAL_CME_HARP_ASSOCIATIONS', 'MAJUMDAR_SRC']

for table in tables_to_update:
    for old_id, new_id in translation_dict.items():
        cursor.execute(f"UPDATE {table} SET cme_id = ? WHERE cme_id = ?", (int(new_id), int(old_id)))

for old_id, new_id in translation_dict.items():
    cursor.execute("UPDATE HARPS_BBOX SET prev_present_at_cme_id = ? WHERE prev_present_at_cme_id = ?", (int(new_id), int(old_id)))
    cursor.execute("UPDATE HARPS_BBOX SET next_present_at_cme_id = ? WHERE next_present_at_cme_id = ?", (int(new_id), int(old_id)))

# Step 5: Re-enable foreign key constraints
cursor.execute("PRAGMA foreign_keys = ON;")

# Commit the changes and close the connection

In [16]:
conn.commit()
conn.close()

In [13]:
conn.close()

In [19]:
# Now to fill the CMEs I dropped because they were duplicates

cmes = pd.read_csv(LASCO_CME_DATABASE)
cmes.replace({np.nan: None}, inplace=True)
no_poor_cmes = cmes[cmes["CME_QUALITY"] == 0]
len(no_poor_cmes)

13311

In [29]:
cmes[cmes.duplicated(subset=["CME_ID"], keep=False)]

,CME_ID,CME_DATE,CME_PA,CME_WIDTH,CME_LINEAR_SPEED,CME_2ND_ORDER_INITIAL_SPEED,CME_2ND_ORDER_FINAL_SPEED,CME_2ND_ORDER_20R_SPEED,CME_ACCELERATION,CME_MASS,CME_KINETIC_ENERGY,CME_MPA,CME_HALO,CME_SEEN_IN,CME_QUALITY,CME_THREE_POINTS
232,ID19970208183005,1997-02-08 18:30:05,325.0,2,379.0,248.0,502.0,699.0,None,None,None,320,0,0,0,0
233,ID19970208183005,1997-02-08 18:30:05,289.0,8,158.0,270.0,53.0,0.0,None,None,None,281,0,0,1,3
368,ID19970710172603,1997-07-10 17:26:03,256.0,30,91.0,105.0,76.0,0.0,None,None,None,263,0,0,1,0
369,ID19970710172603,1997-07-10 17:26:03,91.0,52,209.0,87.0,328.0,898.0,None,None,None,89,0,1,1,3
517,ID19971105121042,1997-11-05 12:10:42,302.0,15,603.0,601.0,605.0,612.0,None,None,None,300,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32464,ID20220715082405,2022-07-15 08:24:05,273.0,18,277.0,115.0,434.0,1219.0,None,None,None,272,0,1,2,0
32465,ID20220715082405,2022-07-15 08:24:05,72.0,28,386.0,411.0,362.0,0.0,None,None,None,76,0,1,1,0
32521,ID20220729123636,2022-07-29 12:36:36,113.0,12,323.0,231.0,419.0,999.0,None,None,None,115,0,1,1,0
32522,ID20220729123636,2022-07-29 12:36:36,319.0,22,425.0,220.0,620.0,1251.0,None,None,None,319,0,1,1,0


In [35]:
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

not_matched = []

for i, row in cmes.iterrows():
    cme_pa = f"{int(row['CME_PA']):03}" if not pd.isna(row["CME_PA"]) else "999"
    cme_id = str(row["CME_ID"][2:]) + cme_pa

    cur.execute("SELECT cme_id FROM CMES WHERE cme_id = ?", (int(cme_id),))

    matches = cur.fetchall()

    exists = True if len(matches) > 0 else False

    if len(matches) > 1:
        raise ValueError("More than one match")

    if not exists:
        cur.execute(
            """
            INSERT INTO cmes (cme_id, cme_date, cme_pa, cme_width, cme_halo, cme_seen_in, cme_quality, cme_three_points)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?)
            """,
            (
                cme_id,
                row["CME_DATE"],
                row["CME_PA"],
                row["CME_WIDTH"],
                row["CME_HALO"],
                row["CME_SEEN_IN"],
                row["CME_QUALITY"],
                row["CME_THREE_POINTS"]
            )
        )

In [36]:
conn.commit()
conn.close()

In [58]:
# And I need to update their image_timestamps
from datetime import datetime
import bisect
from tqdm import tqdm

conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

image_timestamps = cur.execute("SELECT timestamp FROM images").fetchall()
image_timestamps = [t[0] for t in image_timestamps]
timestamps = sorted([datetime.strptime(t, "%Y-%m-%d %H:%M:%S") for t in image_timestamps])

new_added = cur.execute("SELECT cme_id FROM CMES WHERE image_timestamp IS NULL").fetchall()

# Function to find the closest timestamp using binary search
def closest_timestamp(target, sorted_timestamps):
    index = bisect.bisect_left(sorted_timestamps, target)
    if index == 0:
        return sorted_timestamps[0]
    if index == len(sorted_timestamps):
        return sorted_timestamps[-1]
    before = sorted_timestamps[index - 1]
    after = sorted_timestamps[index]
    if after - target < target - before:
       return after
    else:
       return before

for row in tqdm(cur.execute("SELECT cme_id, cme_date FROM CMES WHERE image_timestamp IS NULL").fetchall()):
    cme_timestamp = datetime.strptime(row[1], "%Y-%m-%d %H:%M:%S")
    image_timestamp = closest_timestamp(cme_timestamp, timestamps)
    cur.execute("UPDATE CMES SET image_timestamp = ? WHERE cme_id = ?", (image_timestamp.strftime("%Y-%m-%d %H:%M:%S"), row[0]))

100%|██████████| 742/742 [00:00<00:00, 25037.20it/s]


In [59]:
conn.commit()
conn.close()

In [50]:
new_added = [new[0] for new in new_added]

In [54]:
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

cur.execute("""CREATE TABLE IF NOT EXISTS NEW_ADDED_CMES (
            cme_id INTEGER PRIMARY KEY REFERENCES CMES(cme_id)
)""")

for new in new_added:
    cur.execute("INSERT INTO NEW_ADDED_CMES (cme_id) VALUES (?);", (new,))

In [57]:
conn.commit()
conn.close()

In [23]:
# Now I've rerun the whole catalogue with the added CMEs (mind that I dropped 4 of them which had the exact same time and PA but different widths)
# Let's do some sanity checks

import sys
import sqlite3
import pandas as pd
from IPython.display import display
sys.path.append("../")
from src.cmesrc.config import LASCO_CME_DATABASE, CMESRCV2_DB, SPATIOTEMPORAL_MATCHING_HARPS_DATABASE_PICKLE, DIMMINGS_MATCHED_TO_HARPS_PICKLE, FLARES_MATCHED_TO_HARPS_PICKLE

In [16]:
# Check all CMEs are in database
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

df = pd.read_csv(LASCO_CME_DATABASE)

# Now look at the duplicated
duplicated = df[df.duplicated(subset=["CME_ID"], keep=False)]
print("DUPLICATED ARE")
display(duplicated)

df = df[~df.duplicated(subset=["CME_ID"], keep=False)]

# Now check all CMEs are in database

non_in_database = []

for cme_id in df["CME_ID"]:
    cur.execute("SELECT COUNT(*) FROM CMES WHERE cme_id = ?", (int(cme_id[2:]),))

    if cur.fetchone()[0] == 0:
        print(f"{cme_id} not in database")
        non_in_database.append(cme_id)

if len(non_in_database) == 0:
    print("All CMEs in database!")
else:
    print("Not all CMEs in database!")

conn.close()

DUPLICATED ARE


,CME_ID,CME_DATE,CME_PA,CME_WIDTH,CME_LINEAR_SPEED,CME_2ND_ORDER_INITIAL_SPEED,CME_2ND_ORDER_FINAL_SPEED,CME_2ND_ORDER_20R_SPEED,CME_ACCELERATION,CME_MASS,CME_KINETIC_ENERGY,CME_MPA,CME_HALO,CME_SEEN_IN,CME_QUALITY,CME_THREE_POINTS
11034,ID20060517230604238,2006-05-17 23:06:04,238.0,34,594.0,633.0,552.0,574.0,NaN,1.300000e+16,2.300000e+31,231,0,0,0,0
11035,ID20060517230604238,2006-05-17 23:06:04,238.0,42,309.0,167.0,467.0,593.0,NaN,1.000000e+15,4.900000e+29,233,0,0,0,0
17203,ID20110918112407076,2011-09-18 11:24:07,76.0,14,175.0,176.0,174.0,157.0,NaN,NaN,NaN,69,0,1,2,0
17204,ID20110918112407076,2011-09-18 11:24:07,76.0,16,168.0,178.0,157.0,0.0,NaN,NaN,NaN,71,0,1,2,0


All CMEs in database!


In [20]:
# Ok, now we need to check the spatiotemporal associations before we add the new ones
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

df = pd.read_pickle(SPATIOTEMPORAL_MATCHING_HARPS_DATABASE_PICKLE)
df = df[df["HARPS_SPAT_CONSIST"]]

new_cmes = cur.execute("SELECT cme_id FROM NEW_ADDED_CMES").fetchall()
new_cmes = [int(cme_id[0]) for cme_id in new_cmes]

display(df.head())

non_added_matches = []

for cme_id, harpnum in df[["CME_ID", "HARPNUM"]].values:
    cme_id = int(cme_id[2:])
    harpnum = int(harpnum)
    if cme_id in new_cmes:
        continue
    else:
        cur.execute("SELECT COUNT(*) FROM CMES_HARPS_SPATIALLY_CONSIST WHERE cme_id = ? AND harpnum = ?", (cme_id, harpnum))
        count = cur.fetchone()[0]

        if count == 0:
            non_added_matches.append((cme_id, harpnum))
            print(f"New match found: {cme_id} - {harpnum}")
if len(non_added_matches) == 0:
    print("No new matches found")
else:
    print(f"Found {len(non_added_matches)} new matches")

conn.close()

,CME_ID,CME_HARPNUM_ID,CME_DATE,HARPNUM,CME_PA,CME_WIDTH,CME_LINEAR_SPEED,CME_2ND_ORDER_INITIAL_SPEED,CME_2ND_ORDER_FINAL_SPEED,CME_2ND_ORDER_20R_SPEED,...,HARPS_MIDPOINT,HARPS_DISTANCE_TO_SUN_CENTRE,HARPS_PA,CME_HARPS_PA_DIFF,HARPS_RAW_BBOX,HARPS_LONDTMIN,HARPS_LATDTMIN,HARPS_LONDTMAX,HARPS_LATDTMAX,HARPS_SPAT_CONSIST
CME_HARPNUM_ID,,,,,,,,,,,,,,,,,,,,,
ID201005010048051011,ID20100501004805101,ID201005010048051011,2010-05-01 00:48:05.000,1,101.0,126,408.0,227.0,591.0,856.0,...,"[-78.624901, 24.490599500000002]",0.98376,65.07781,35.92219,"[[-87.321999, 20.513599], [-69.927803, 28.4676]]",-87.321999,20.513599,-69.927803,28.4676,True
ID201005010048051012,ID20100501004805101,ID201005010048051012,2010-05-01 00:48:05.000,2,101.0,126,408.0,227.0,591.0,856.0,...,"[-28.11575, 14.2224005]",0.518688,61.727283,39.272717,"[[-33.939201, 9.9477], [-22.292299, 18.497101]]",-33.939201,9.9477,-22.292299,18.497101,True
ID201005010712051111,ID20100501071205111,ID201005010712051111,2010-05-01 07:12:05.000,1,111.0,108,360.0,338.0,384.0,493.0,...,"[-75.0973015, 24.490599500000002]",0.972227,64.761579,46.238421,"[[-83.794403, 20.513599], [-66.4002, 28.4676]]",-83.794403,20.513599,-66.4002,28.4676,True
ID201005010712051112,ID20100501071205111,ID201005010712051112,2010-05-01 07:12:05.000,2,111.0,108,360.0,338.0,384.0,493.0,...,"[-24.5182, 14.2224005]",0.471357,58.585047,52.414953,"[[-30.3417, 9.9477], [-18.6947, 18.497101]]",-30.3417,9.9477,-18.6947,18.497101,True
ID201005021930052465,ID20100502193005246,ID201005021930052465,2010-05-02 19:30:05.000,5,246.0,64,242.0,52.0,430.0,367.0,...,"[31.5621005, -28.7032995]",0.664409,223.708957,22.291043,"[[26.7087, -30.195], [36.415501, -27.211599]]",26.7087,-30.195,36.415501,-27.211599,True


No new matches found


In [22]:
# So since we agree that no new matches are found for existing CMES, the script hasn't broken and we can add the new matches

conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

df = pd.read_pickle(SPATIOTEMPORAL_MATCHING_HARPS_DATABASE_PICKLE)
df = df[df["HARPS_SPAT_CONSIST"]]

new_cmes = cur.execute("SELECT cme_id FROM NEW_ADDED_CMES").fetchall()
new_cmes = [int(cme_id[0]) for cme_id in new_cmes]

display(df.head())

new_matches = []

for cme_id, harpnum in df[["CME_ID", "HARPNUM"]].values:
    cme_id = int(cme_id[2:])
    harpnum = int(harpnum)
    if cme_id in new_cmes:
        # Check it doesn
        cur.execute("INSERT OR IGNORE INTO CMES_HARPS_SPATIALLY_CONSIST (harpnum, cme_id) VALUES (?, ?)", (harpnum, cme_id))
        new_matches.append((harpnum, cme_id))

conn.commit()
conn.close()
print(f"Added {len(new_matches)} new matches")

,CME_ID,CME_HARPNUM_ID,CME_DATE,HARPNUM,CME_PA,CME_WIDTH,CME_LINEAR_SPEED,CME_2ND_ORDER_INITIAL_SPEED,CME_2ND_ORDER_FINAL_SPEED,CME_2ND_ORDER_20R_SPEED,...,HARPS_MIDPOINT,HARPS_DISTANCE_TO_SUN_CENTRE,HARPS_PA,CME_HARPS_PA_DIFF,HARPS_RAW_BBOX,HARPS_LONDTMIN,HARPS_LATDTMIN,HARPS_LONDTMAX,HARPS_LATDTMAX,HARPS_SPAT_CONSIST
CME_HARPNUM_ID,,,,,,,,,,,,,,,,,,,,,
ID201005010048051011,ID20100501004805101,ID201005010048051011,2010-05-01 00:48:05.000,1,101.0,126,408.0,227.0,591.0,856.0,...,"[-78.624901, 24.490599500000002]",0.98376,65.07781,35.92219,"[[-87.321999, 20.513599], [-69.927803, 28.4676]]",-87.321999,20.513599,-69.927803,28.4676,True
ID201005010048051012,ID20100501004805101,ID201005010048051012,2010-05-01 00:48:05.000,2,101.0,126,408.0,227.0,591.0,856.0,...,"[-28.11575, 14.2224005]",0.518688,61.727283,39.272717,"[[-33.939201, 9.9477], [-22.292299, 18.497101]]",-33.939201,9.9477,-22.292299,18.497101,True
ID201005010712051111,ID20100501071205111,ID201005010712051111,2010-05-01 07:12:05.000,1,111.0,108,360.0,338.0,384.0,493.0,...,"[-75.0973015, 24.490599500000002]",0.972227,64.761579,46.238421,"[[-83.794403, 20.513599], [-66.4002, 28.4676]]",-83.794403,20.513599,-66.4002,28.4676,True
ID201005010712051112,ID20100501071205111,ID201005010712051112,2010-05-01 07:12:05.000,2,111.0,108,360.0,338.0,384.0,493.0,...,"[-24.5182, 14.2224005]",0.471357,58.585047,52.414953,"[[-30.3417, 9.9477], [-18.6947, 18.497101]]",-30.3417,9.9477,-18.6947,18.497101,True
ID201005021930052465,ID20100502193005246,ID201005021930052465,2010-05-02 19:30:05.000,5,246.0,64,242.0,52.0,430.0,367.0,...,"[31.5621005, -28.7032995]",0.664409,223.708957,22.291043,"[[26.7087, -30.195], [36.415501, -27.211599]]",26.7087,-30.195,36.415501,-27.211599,True


Added 351 new matches


In [30]:
# Good! Now, I think that dimmings and flare events shouldn't have changed. Let's check this

flares = pd.read_pickle(FLARES_MATCHED_TO_HARPS_PICKLE)
print("FLARES")
display(flares.head())

if len(flares[flares.duplicated(subset="FLARE_ID", keep=False)]) != 0:
    print("There are duplicated flare events")
    display(flares[flares.duplicated(subset="FLARE_ID", keep=False)])

# Same for dimmings

dimmings = pd.read_pickle(DIMMINGS_MATCHED_TO_HARPS_PICKLE)
dimmings = dimmings[dimmings["MATCH"]]
print("DIMMINGS")
display(dimmings.head())

if len(dimmings[dimmings.duplicated(subset="dimming_id", keep=False)]) != 0:
    print("There are duplicated dimming events")
    display(dimmings[dimmings.duplicated(subset="dimming_id", keep=False)])

FLARES


,HARPNUM,FLARE_ID,FLARE_DATE,FLARE_LON,FLARE_LAT,FLARE_CLASS_SCORE,FLARE_CLASS,FLARE_AR,FLARE_AR_SOURCE,FLARE_VERIFICATION
0,1979,4529,2012-08-27 00:12:00.000,-87.553307,14.033801,15.5,B5.5,11558,GOES,Primary
1,1979,4532,2012-08-27 08:12:00.000,-85.542824,15.844800,19.8,B9.8,11558,GOES,Primary
2,1979,4534,2012-08-27 15:12:00.000,-81.081421,16.128628,14.1,B4.1,11558,GOES,Primary
3,1979,4535,2012-08-27 16:12:00.000,-80.549965,16.142371,15.6,B5.6,11558,GOES,Primary
4,1979,4536,2012-08-27 17:48:00.000,-79.746639,16.203563,14.8,B4.8,11558,GOES,Primary


DIMMINGS


,dimming_id,start_time,end_time,last_detection_time,half_value_time,max_detection_time,sequences,max_detection_ratio,f_r,avg_x,...,HARPNUM,DIMMING_HARPNUM_ID,HARPS_RAW_LONDTMIN,HARPS_RAW_LATDTMIN,HARPS_RAW_LONDTMAX,HARPS_RAW_LATDTMAX,HARPS_RAW_DATE,HARPS_DIMMING_DISTANCE,POSITION_SCORES,MATCH
0,4321,2010-05-23 16:46:02.0,2010-05-23 17:48:02.0,2010-05-23 17:24:02.0,2010-05-23 16:46:02.0,2010-05-23 16:46:02.000,40,1.198048,0.504325,-0.019409,...,24,ID432124,1.143900,12.851200,15.800900,19.236099,2010-05-23 16:48:00.000,0.166122,8.112107,True
9,4322,2010-05-24 13:22:02.0,2010-05-24 13:52:02.0,2010-05-24 13:28:02.0,2010-05-24 13:22:02.0,2010-05-24 13:24:02.000,24,1.784743,0.611199,0.525675,...,25,ID432225,23.213100,18.135799,31.947599,22.745199,2010-05-24 13:24:00.000,0.026837,93.654685,True
12,4323,2010-05-31 19:38:02.0,2010-05-31 20:34:02.0,2010-05-31 20:10:02.0,2010-05-31 19:38:02.0,2010-05-31 19:54:02.000,37,3.101171,0.615685,0.456310,...,36,ID432336,33.858898,18.482800,41.045101,23.133400,2010-05-31 20:00:00.000,0.077304,58.047253,True
17,4324,2010-06-12 01:04:02.0,2010-06-12 01:50:02.0,2010-06-12 01:26:02.0,2010-06-12 01:04:02.0,2010-06-12 01:08:02.000,32,39.170922,0.845090,0.711072,...,54,ID432454,37.626598,19.711901,58.693600,27.378599,2010-06-12 01:12:00.000,0.023543,95.080325,True
20,4325,2010-06-12 09:28:02.0,2010-06-12 09:52:02.0,2010-06-12 09:28:02.0,2010-06-12 09:28:02.0,2010-06-12 09:28:02.000,21,3.614824,0.882495,0.762432,...,54,ID432554,42.157398,19.711901,63.224400,27.378599,2010-06-12 09:24:00.000,0.000000,100.000000,True


In [32]:
# Now the actual checking
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

non_matched_flares = []

for flare_id, harpnum in flares[["FLARE_ID", "HARPNUM"]].values:
    flare_id = int(flare_id)
    harpnum = int(harpnum)
    cur.execute("SELECT COUNT(*) FROM FLARES WHERE flare_id = ? and harpnum = ?", (flare_id, harpnum))

    count = cur.fetchone()[0]

    if count == 0:
        print(f"Flare {flare_id} with HARP {harpnum} not found in database.")
        non_matched_flares.append(flare_id)

if len(non_matched_flares) == 0:
    print("All flares matched.")
else:
    print(f"{len(non_matched_flares)} flares not matched.")

conn.close()

All flares matched.


In [35]:

# Now the actual checking
conn = sqlite3.connect(CMESRCV2_DB)
cur = conn.cursor()

non_matched_dimmings = []

for dimming_id, harpnum in dimmings[["dimming_id", "HARPNUM"]].values:
    dimming_id = int(dimming_id)
    harpnum = int(harpnum)
    cur.execute("SELECT COUNT(*) FROM DIMMINGS WHERE dimming_id = ? and harpnum = ?", (dimming_id, harpnum))

    count = cur.fetchone()[0]

    if count == 0:
        print(f"Dimming {dimming_id} with HARP {harpnum} not found in database.")
        non_matched_dimmings.append(dimming_id)

if len(non_matched_dimmings) == 0:
    print("All dimmings matched.")
else:
    print(f"{len(non_matched_dimmings)} dimmings not matched.")

conn.close()

All dimmings matched.


In [ ]:
# ALL DONE!